# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [64]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [65]:
weather_df = pd.read_csv("cities.csv")
weather_data_df = weather_df.dropna(how="any")
del weather_data_df["Unnamed: 0"]
weather_data_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,atuona,-46.19,168.86,66.00,49.0,86.0,5.01,NZ,1.603330e+09
1,busselton,-9.80,-139.03,77.95,71.0,0.0,16.26,PF,1.603330e+09
2,alamosa,-33.65,115.33,60.01,66.0,9.0,3.00,AU,1.603330e+09
3,port hawkesbury,37.47,-105.87,57.20,12.0,1.0,13.87,US,1.603330e+09
4,sao filipe,45.62,-61.35,54.00,100.0,90.0,9.17,CA,1.603330e+09
5,broken hill,14.90,-24.50,81.16,74.0,64.0,4.79,CV,1.603330e+09
6,new norfolk,-31.95,141.43,80.60,18.0,0.0,10.29,AU,1.603330e+09
7,mufumbwe,-42.78,147.06,66.00,49.0,10.0,1.99,AU,1.603330e+09
8,amderma,-13.68,24.80,66.87,50.0,57.0,2.51,ZM,1.603330e+09
11,lima,57.79,-152.41,46.40,52.0,1.0,12.75,US,1.603329e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [66]:
coord = weather_data_df[["Lat", "Lng"]].astype(float)
weight = weather_data_df["Humidity"].astype(float)

In [67]:
fig=gmaps.figure()
heat_layer=gmaps.heatmap_layer(coord, weights=weight, 
                               dissipating=False, max_intensity=10,
                              point_radius=2)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [68]:
hotel_df = weather_data_df.loc[(weather_data_df["Max Temp"]>70) & (weather_data_df["Max Temp"]<80) 
                                    & (weather_data_df["Wind Speed"]<10) & (weather_data_df["Cloudiness"]==0)]
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
17,illoqqortoormiut,25.25,78.47,71.28,29.0,0.0,2.26,IN,1.603330e+09
63,yumen,10.40,5.47,74.84,60.0,0.0,0.81,NG,1.603330e+09
94,matay,21.43,39.83,73.33,23.0,0.0,1.74,SA,1.603330e+09
95,acajutla,28.42,30.78,70.68,69.0,0.0,6.96,EG,1.603330e+09
125,tautira,28.07,95.33,72.57,72.0,0.0,3.38,IN,1.603330e+09
216,ati,21.27,-97.45,76.78,84.0,0.0,4.18,MX,1.603330e+09
343,faya,-8.93,125.41,78.62,43.0,0.0,1.10,TL,1.603330e+09
372,siparia,5.88,-57.13,79.63,83.0,0.0,7.83,GY,1.603330e+09
407,ravar,31.61,34.76,71.01,77.0,0.0,3.36,IL,1.603330e+09
419,sola,-2.97,-47.48,79.70,63.0,0.0,6.08,BR,1.603330e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [69]:
hotel_df["Hotel Name"] = ""
hotel_df.head()

<ipython-input-69-64598bec773a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
17,illoqqortoormiut,25.25,78.47,71.28,29.0,0.0,2.26,IN,1.603330e+09,
63,yumen,10.40,5.47,74.84,60.0,0.0,0.81,NG,1.603330e+09,
94,matay,21.43,39.83,73.33,23.0,0.0,1.74,SA,1.603330e+09,
95,acajutla,28.42,30.78,70.68,69.0,0.0,6.96,EG,1.603330e+09,
125,tautira,28.07,95.33,72.57,72.0,0.0,3.38,IN,1.603330e+09,


In [70]:
def nearby(params):
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    data = requests.get(base_url, params=params).json()
    return data

params = {
    'key' : g_key,
    'radius' : 5000,
    'type' : 'Hotel'}

for index, row in hotel_df.iterrows():
    Lat= row["Lat"]
    Lng= row["Lng"]
    params['location']=f"{Lat},{Lng}"
    data=nearby(params)
    
    try:
        hotel_df.loc[index, "Hotel Name"] = data['results'][0]['name']
    except:
        print(f"No hotel near this {City}")
        pass
    
hotel_df

/Users/sarah/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
17,illoqqortoormiut,25.25,78.47,71.28,29.0,0.0,2.26,IN,1.603330e+09,Babina
63,yumen,10.40,5.47,74.84,60.0,0.0,0.81,NG,1.603330e+09,Kontagora
94,matay,21.43,39.83,73.33,23.0,0.0,1.74,SA,1.603330e+09,Mecca
95,acajutla,28.42,30.78,70.68,69.0,0.0,6.96,EG,1.603330e+09,Matay
125,tautira,28.07,95.33,72.57,72.0,0.0,3.38,IN,1.603330e+09,Nyingchi
216,ati,21.27,-97.45,76.78,84.0,0.0,4.18,MX,1.603330e+09,Tamiahua
343,faya,-8.93,125.41,78.62,43.0,0.0,1.10,TL,1.603330e+09,Atsabe
372,siparia,5.88,-57.13,79.63,83.0,0.0,7.83,GY,1.603330e+09,Corriverton
407,ravar,31.61,34.76,71.01,77.0,0.0,3.36,IL,1.603330e+09,Kiryat Gat
419,sola,-2.97,-47.48,79.70,63.0,0.0,6.08,BR,1.603330e+09,Nova Conquista


In [71]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [72]:
fig=gmaps.figure()
heat_layer=gmaps.heatmap_layer(coord, weights=weight, 
                               dissipating=True, max_intensity=20,
                              point_radius=3)
fig.add_layer(heat_layer)

# Add marker layer ontop of heat map

markers=gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))